## Задача оптимизации площади взлетно-посадочной полосы
Рассматривается задача минимизации проектной площади взлетно-посадочной полосы при ограничении на вероятность успешной посадки.
Под случайным внешним воздействием подразумевается смещение летательного аппарата (ЛА) относительно расчетной точки под воздействием ветра.

Обозначения:

$l_0$ - длина пробега ЛА по ВПП до полной остановки

$l_1$ - запас по длиней ВПП на случай недолета до расчетной точки

$l_2$ - запас по длиней ВПП на случай перелета расчетной точки

$z_1$ - полуширина полосы

Тогда площадь ВПП будет определяться соотношением
$$
S = 2z_1(l_0+l_1+l_2)
$$

В расчетах использованы такие значения параметров:

$a_{11}=a_{12}=-20$, $a_{22}=3$, $l_0=1500$

### Модель случайных возмущений
Пусть $X$ и $Z$ случайные отклонения от расчетной точки касания ВПП вдоль и поперек полосы соотетственно. Эти отклонения связаны с компонентами скорости ветра $W_x$ $W_z$
$$
X = a_{11}W_x + a_{12}|W_z|, Z = a_{22}W_z
$$
$W_x, W_z$ - независимые гауссовские случайные величины
$$
E(W_x) = m_x, E(W_z) = m_z, D(W_x) = \sigma^2_x, D(W_z) = \sigma^2_z
$$
В расчетах использованы значения $\sigma_x=\sigma_z=5$, математические ожидания равны 0.

## Критерий успешной посадки
Посадка считается успешной, если
$$
-l_1\le X\le l_2, |Z|\le z_1
$$

## Итоговая задача оптимизации
Для оптимизации будет использоваться градиентный спуск. Он чувствителен к масштабу параметров, получим большой перекос в сторону сужения ширины полосы. Вместо этого перейдем к логарифму площади.
$$
ln(z_1) + ln(l_0+l_1+l_2) \to \min
$$
При ограничении
$$
P(-l_1\le X\le l_2, |Z|\le z_1) \ge \alpha
$$
Для расчетов $\alpha$ полагалась равной 0.99

In [18]:
from LFlow.stochastic_model import IndependentGenerator, MultivariateGaussGenerator, stats
from LFlow.labos_flow_v2 import LabFunc, np, plt, LabSigmoid, LabLog, Identity
from LFlow.labos_point import Point
#np.seterr(all='ignore')

alpha = 0.99
max_iter = 150
step = 100#0.01
penalty = 10

base_point = Point({'a1' : -20, 'a2' : -20, 'a3' : 3})

l1 = Identity('l1')
l2 = Identity('l2')
z1 = Identity('z1')

# take logarithm of halved area
area = LabLog(2*l1+2*l2) + LabLog(z1)
X = LabFunc('a1*Wx + a2*np.abs(Wz)', derivatives={'a1' : 'Wx', 'a2' : 'np.abs(Wz)'}, args=['a1', 'a2', 'Wx', 'Wz'])
Z = LabFunc('a3*np.abs(Wz)', derivatives={'a3' : 'np.abs(Wz)'}, args=['a3', 'Wz'])

# to compare with original solution
u1 = l1/l2
u2 = 0.5*(1500 + l1 + l2)/z1
S = 2*(1500 + l1 + l2)*z1

# must be positive
theta=10
cond1 = LabSigmoid((X+l1)*(l2-X), theta=theta)
cond2 = LabSigmoid((z1**2) - (Z**2), theta=theta)
total_cond = cond1*cond2

start_point = base_point.expand(Point({'l1' : 600, 'l2' : 300, 'z1' : 60}))

sm = MultivariateGaussGenerator(['Wx', 'Wz'], cov=[[25, 0], [0, 25]])
sample_points, sample_dicts = sm.rvs(5000)

Находим стартовую точку

In [19]:
cond_mean, cond_gradient = sm.papa_carlo(total_cond, start_point, sample_dicts, derivs = ['l1', 'l2', 'z1'])
while cond_mean < alpha + 0.5*(1-alpha):
    """
    update start point
    0.5*(1-alpha) used to dive deeper in region
    """
    print(cond_mean)
    start_point['l1'] = start_point['l1'] + 1
    start_point['l2'] = start_point['l2'] + 1
    start_point['z1'] = start_point['z1'] + 1
    
    cond_mean, cond_gradient = sm.papa_carlo(total_cond, start_point, sample_dicts, derivs = ['l1', 'l2', 'z1'])
    
crit = area(start_point)
crit_gradient = area.deriv(start_point)

In [20]:
opt_crit = crit
opt_point = start_point
for i in range(max_iter):
    if i % 20 == 0:
        print('step {}'.format(i))
        print('l1 {:.1f} l2 {:.1f} z1 {:.1f} area {:.2e} prob {:.3f}'.format(start_point['l1'],
                                                                             start_point['l2'],
                                                                             start_point['z1'],
                                                                             S(start_point), cond_mean))

    if start_point['z1'] - step*crit_gradient['z1'] < 0:
        factor = abs(0.5*start_point['z1']/(step*crit_gradient['z1']))
    else:
        factor = 1
    """
    dz1 cannot exceed 2, just because i want it..
    the derivative with respect to z1 is always big
    """
    factor = min(factor, 2/(abs(step*factor*crit_gradient['z1'])))
    cond_mean, cond_grad = sm.papa_carlo(total_cond, start_point - step*factor*crit_gradient, sample_dicts, derivs=['l1', 'l2', 'z1'])
    if cond_mean < alpha:
        factor = 0.8*factor
        cond_mean, cond_grad = sm.papa_carlo(total_cond, start_point - step*factor*crit_gradient, sample_dicts, derivs=['l1', 'l2', 'z1'])
        if cond_mean > alpha:
            start_point = start_point - step*factor*crit_gradient
        else:
            """
            gradient projection
            """
            direction = crit_gradient - cond_grad*cond_grad.cos(crit_gradient)*crit_gradient.norm()/cond_grad.norm()
            start_point = start_point - step*direction
            cond_mean, cond_grad = sm.papa_carlo(total_cond, start_point, sample_dicts, derivs=['l1', 'l2', 'z1'])
            while cond_mean < alpha:
                """
                если после шага вдоль ограничений попали вне допустимой области
                """
                start_point = start_point + factor*cond_grad
                cond_mean, cond_grad = sm.papa_carlo(total_cond, start_point, sample_dicts, derivs=['l1', 'l2', 'z1'])
    else:
        start_point = start_point - step*factor*crit_gradient
    crit = area(start_point)
    crit_gradient = area.deriv(start_point)
    if crit < opt_crit:
        opt_crit = crit
        opt_point = Point(start_point.dict.copy())
print('finish')
print('l1 {:.1f} l2 {:.1f} z1 {:.1f} area {:.2e}'.format(opt_point['l1'], opt_point['l2'], opt_point['z1'], S(opt_point)))
print('u1 {:.3f}'.format(u1(opt_point)))
print('u2 {:.3f}'.format(u2(opt_point)))

step 0
l1 600.0 l2 300.0 z1 60.0 area 2.88e+05 prob 1.000
step 20
l1 597.8 l2 297.8 z1 39.2 area 1.88e+05 prob 0.990
step 40
l1 595.6 l2 295.6 z1 39.2 area 1.87e+05 prob 0.990
step 60
l1 593.3 l2 293.3 z1 39.2 area 1.87e+05 prob 0.990
step 80
l1 591.1 l2 291.1 z1 39.2 area 1.87e+05 prob 0.990
step 100
l1 588.8 l2 288.8 z1 39.2 area 1.86e+05 prob 0.990
step 120
l1 586.5 l2 286.5 z1 39.2 area 1.86e+05 prob 0.990
step 140
l1 584.2 l2 284.2 z1 39.2 area 1.86e+05 prob 0.990
finish
l1 583.1 l2 283.1 z1 39.2 area 1.85e+05
u1 2.060
u2 30.189
